### Topic Modeling using HDP and LDA

- Text Processing
- Generating dictionary of vocabulary
- Mapping corpus using dictionary
- Training the Topic Model

In [2]:
import matplotlib.pyplot as plt
import gensim
import numpy as np
import spacy
import pandas as pd
import re

from gensim import corpora, models
from gensim.models.ldamodel import LdaModel
from spacy.lang.en.stop_words import STOP_WORDS
import pyLDAvis.gensim
#Import nltk stopwords and add custom stopwords that are likely to appear in news articles.
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(["mrs","ms","say","he","mr","she","they","company"])

import os, re, operator, warnings
warnings.filterwarnings('ignore')
%matplotlib inline

C:\ProgramData\Anaconda3\lib\site-packages\past\types\oldstr.py:33: DeprecationWarning: invalid escape sequence \d
  """
C:\ProgramData\Anaconda3\lib\site-packages\nltk\decorators.py:68: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  regargs, varargs, varkwargs, defaults, formatvalue=lambda value: ""
C:\ProgramData\Anaconda3\lib\site-packages\nltk\featstruct.py:1296: DeprecationWarning: invalid escape sequence \d
  name, n = re.sub('\d+$', '', var.name), 2
C:\ProgramData\Anaconda3\lib\site-packages\nltk\featstruct.py:2092: DeprecationWarning: invalid escape sequence \d
  RANGE_RE = re.compile('(-?\d+):(-?\d+)')
C:\ProgramData\Anaconda3\lib\site-packages\nltk\sem\evaluate.py:311: DeprecationWarning: invalid escape sequence \ 
  """
C:\ProgramData\Anaconda3\lib\site-packages\nltk\sem\relextract.py:135: DeprecationWarning: invalid escape sequence \w
  ENT = re.compile("&(\w+?);")
C:\ProgramData\Anaconda3\lib\site-p

C:\ProgramData\Anaconda3\lib\site-packages\nltk\chunk\regexp.py:718: DeprecationWarning: invalid escape sequence \{
  '(?P<left>%s)\{(?P<right>%s)'
C:\ProgramData\Anaconda3\lib\site-packages\nltk\chunk\regexp.py:724: DeprecationWarning: invalid escape sequence \g
  RegexpChunkRule.__init__(self, regexp, '{\g<left>\g<right>', descr)
C:\ProgramData\Anaconda3\lib\site-packages\nltk\chunk\regexp.py:789: DeprecationWarning: invalid escape sequence \}
  '(?P<left>%s)\}(?P<right>%s)'
C:\ProgramData\Anaconda3\lib\site-packages\nltk\chunk\regexp.py:795: DeprecationWarning: invalid escape sequence \g
  RegexpChunkRule.__init__(self, regexp, '\g<left>\g<right>}', descr)
C:\ProgramData\Anaconda3\lib\site-packages\nltk\chunk\regexp.py:908: DeprecationWarning: invalid escape sequence \{
  r'^((%s|<%s>)*)$' % ('([^\{\}<>]|\{\d+,?\}|\{\d*,\d+\})+', '[^\{\}<>]+')
C:\ProgramData\Anaconda3\lib\site-packages\nltk\chunk\regexp.py:908: DeprecationWarning: invalid escape sequence \{
  r'^((%s|<%s>)*)$' % ('(

### Text Processing
- Clean the article - Remove punctuation marks, special characters
- Tokenize each article
- Stem each token
- Remove numberical tokens

In [3]:
df=pd.read_csv("NewsArticles.csv", encoding='unicode_escape',index_col=0)
#drop all the unnamed columns
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df.head()

,publish_date,article_source_link,title,subtitle,text
article_id,,,,,
1,2017/2/7,http://abcnews.go.com/Politics/pence-break-tie...,"Betsy DeVos Confirmed as Education Secretary, ...",NaN,Michigan billionaire education activist Betsy ...
2,2017/2/7,http://abcnews.go.com/Politics/wireStory/melan...,Melania Trump Says White House Could Mean Mill...,NaN,First lady Melania Trump has said little about...
3,2017/2/7,http://abcnews.go.com/Politics/wireStory/trump...,"As Trump Fears Fraud, GOP Eliminates Election ...",NaN,A House committee voted on Tuesday to eliminat...
4,2017/2/7,http://abcnews.go.com/Politics/appeals-court-d...,Appeals Court to Decide on Challenge to Trump'...,NaN,"This afternoon, three federal judges from the ..."
5,2017/2/7,http://abcnews.go.com/US/23-states-winter-weat...,At Least 4 Tornadoes Reported in Southeast Lou...,NaN,At least four tornadoes touched down in Louisi...


I'm going to use SPACY in this notebook for all the text processing related tasks. It is very powerful than NLTK. [Click here to learn more](https://spacy.io/usage/spacy-101)

In [4]:
# before loading the language you have to download it first. Go to your command prompt and execute this statement and 
# restart the kernel:
# python -m spacy download en_core_web_sm
nlp = spacy.load("en_core_web_sm")

In [5]:
data = df['text'].values.tolist()

In [5]:
#removing punctuations and others characters
def preprocess(string):
    return re.sub('[^\w_\s-]', ' ',str(string))

data = list(map(preprocess,data))    

In [6]:
#data cleaning and lemmatization
lemma_doc = []
for datum in data:
    sent = nlp(str(datum).lower())
    text = []
    for w in sent:
        if not w.is_stop and not w.is_punct and not w.like_num and str(w) not in stop_words and (len(str(w)) > 4):
            #adding the lematized version of the words
            text.append(w.lemma_)
    lemma_doc.append(text)
    

In [7]:
lemma_doc[0]

['michigan',
 'billionaire',
 'education',
 'activist',
 'betsy',
 'devos',
 'confirm',
 'today',
 'serve',
 'secretary',
 'education',
 'president',
 'trump',
 'administration',
 'president',
 'pence',
 'break',
 'senate',
 'senate',
 'vote',
 'devos"?highly',
 'contentious',
 'nomination',
 'afternoon',
 'tally',
 'split',
 'evenly',
 'require',
 'penny',
 'authority',
 'president',
 'upper',
 'chamber',
 'congress',
 'break',
 'impasse',
 'president',
 'break',
 'confirm',
 'cabinet',
 'nominee',
 'pence',
 'count',
 'vote',
 'render',
 'tally',
 'democrats',
 'stage',
 '24-hour',
 'marathon',
 'speech',
 'lawmaker',
 'take',
 'floor',
 'additional',
 'republican',
 'devos',
 'block',
 'confirmation',
 'imagine',
 'bad',
 'choice,"?sen',
 'elizabeth',
 'warren',
 'letter',
 'constituent',
 'urge',
 'devos',
 'stir',
 'vehement',
 'opposition',
 'teachers"?union',
 'senate',
 'democrats',
 'cite',
 'concern',
 'support',
 'school',
 'voucher',
 'critic',
 'believe',
 'weaken',
 'publ

In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(lemma_doc, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[lemma_doc], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[lemma_doc[0]]])

['michigan', 'billionaire', 'education', 'activist', 'betsy_devos', 'confirm', 'today', 'serve', 'secretary', 'education', 'president', 'trump', 'administration', 'president', 'pence', 'break', 'senate', 'senate', 'vote', 'devos"?highly', 'contentious', 'nomination', 'afternoon', 'tally', 'split', 'evenly', 'require', 'penny', 'authority', 'president', 'upper', 'chamber', 'congress', 'break', 'impasse', 'president', 'break', 'confirm', 'cabinet_nominee', 'pence', 'count', 'vote', 'render', 'tally', 'democrats', 'stage', '24-hour', 'marathon', 'speech', 'lawmaker', 'take', 'floor', 'additional', 'republican', 'devos', 'block', 'confirmation', 'imagine', 'bad', 'choice,"?sen', 'elizabeth_warren', 'letter', 'constituent', 'urge', 'devos', 'stir', 'vehement', 'opposition', 'teachers"?union', 'senate', 'democrats', 'cite', 'concern', 'support', 'school', 'voucher', 'critic', 'believe', 'weaken', 'public', 'school', 'experience', 'attend', 'work', 'public', 'education', 'system', 'cite', 'fa

#### Create the Dictionary and Corpus needed for Topic Modeling
- Word to IDs mapping
- Bag of words of each document
- corpus (cluster of Bag of words of all the documents)

In [9]:
#Creates Word to IDs mapping
word2id = corpora.Dictionary(lemma_doc)
print('Let\'s take a look at the dictionary:\n')
print(word2id.token2id)

Let's take a look at the dictionary:



{'"?before': 0, '"?devos': 1, '24-hour': 2, 'account': 3, 'activist': 4, 'addition': 5, 'additional': 6, 'administration': 7, 'administrator': 8, 'afternoon': 9, 'alaska': 10, 'announce': 11, 'answer': 12, 'appreciate': 13, 'attend': 14, 'authority': 15, 'average': 16, 'bad': 17, 'believe': 18, 'betsy': 19, 'billionaire': 20, 'block': 21, 'board': 22, 'break': 23, 'cabinet': 24, 'call': 25, 'caucus': 26, 'chamber': 27, 'charter': 28, 'child': 29, 'choice,"?sen': 30, 'chris': 31, 'cite': 32, 'collin': 33, 'collins': 34, 'committee': 35, 'community': 36, 'concern': 37, 'confirm': 38, 'confirmation': 39, 'congress': 40, 'constituent': 41, 'contentious': 42, 'contribute': 43, 'count': 44, 'critic': 45, 'defend': 46, 'democratic': 47, 'democrats': 48, 'devos': 49, 'devos"?focus': 50, 'devos"?highly': 51, 'devos"?nomination': 52, 'disabled': 53, 'education': 54, 'elizabeth': 55, 'email': 56, 'entire': 57, 'evenly': 58, 'experience': 59, 'familiarity': 60, 'flood': 61, 'floor': 62, 'focus': 6

In [10]:
# Creates bag of words and a corpus
documents = lemma_doc
corpus = [word2id.doc2bow(doc) for doc in documents]

print('Corpus sample')
sample = corpus[0]
for i in range(len(sample)):
    print('Word', sample[i][0], ':', word2id[sample[i][0]], ' || Number of occurences:', sample[i][1])

Corpus sample
Word 0 : "?before  || Number of occurences: 1
Word 1 : "?devos  || Number of occurences: 1
Word 2 : 24-hour  || Number of occurences: 1
Word 3 : account  || Number of occurences: 1
Word 4 : activist  || Number of occurences: 1
Word 5 : addition  || Number of occurences: 1
Word 6 : additional  || Number of occurences: 1
Word 7 : administration  || Number of occurences: 1
Word 8 : administrator  || Number of occurences: 1
Word 9 : afternoon  || Number of occurences: 1
Word 10 : alaska  || Number of occurences: 1
Word 11 : announce  || Number of occurences: 2
Word 12 : answer  || Number of occurences: 1
Word 13 : appreciate  || Number of occurences: 1
Word 14 : attend  || Number of occurences: 1
Word 15 : authority  || Number of occurences: 1
Word 16 : average  || Number of occurences: 1
Word 17 : bad  || Number of occurences: 1
Word 18 : believe  || Number of occurences: 1
Word 19 : betsy  || Number of occurences: 1
Word 20 : billionaire  || Number of occurences: 1
Word 21 

#### Hierarchical Dirichlet Processing
This is kind of an unsupervised technique (Topic modeling is a unsupervised technique. Here the context is we don't decide the # of topics. In concept this is similar to Hierarchical cluster as don't choose the number of cluster before hand) as the model will identify the number of topics. Let's see what it will produce.

In [11]:
hdp = models.HdpModel(corpus,word2id)

In [12]:
hdp_topics = hdp.print_topics()
for topic in hdp_topics:
    print(topic)

(0, '0.008*trump + 0.006*people + 0.005*president + 0.005*country + 0.004*china + 0.004*government + 0.004*state + 0.003*house + 0.003*report + 0.003*year')
(1, '0.006*trump + 0.005*people + 0.005*china + 0.004*country + 0.004*president + 0.003*year + 0.003*government + 0.003*state + 0.003*include + 0.003*party')
(2, '0.007*target + 0.006*china + 0.006*attacker + 0.005*chinese + 0.004*wound + 0.004*attack + 0.004*story + 0.004*kill + 0.003*country + 0.003*police')
(3, '0.006*china + 0.003*government + 0.002*investment + 0.002*president + 0.002*people + 0.002*percent + 0.002*house + 0.002*state + 0.002*daily + 0.002*market')
(4, '0.004*china + 0.002*country + 0.002*people + 0.002*germany + 0.002*world + 0.001*growth + 0.001*price + 0.001*european + 0.001*russian + 0.001*german')
(5, '0.003*child + 0.003*china + 0.002*country + 0.002*percent + 0.002*year + 0.002*people + 0.001*province + 0.001*tsingtao + 0.001*accord + 0.001*report')
(6, '0.004*china + 0.003*chinese + 0.002*jiaozi + 0.00

In [13]:
print('HDP model created: '+str(len(hdp_topics))+' Topics')

HDP model created: 20 Topics


####  Latent Dirichlet Allocation Model


In [14]:
lda_model = LdaModel(corpus=corpus, id2word=word2id, num_topics=5, random_state=42, update_every=1, chunksize=100, 
                     passes=10, alpha='auto', per_word_topics=True)

In [19]:
#Article - Topic Distribution for first Article
def get_article_topic_distribution(article):
    return lda.get_document_topics(article)
#Returns a list containing a list of tuple
#Each inner list corresponds to an article and each tuple refers to topicID and its corresponding probability  
map(get_article_topic_distribution, corpus)

In [15]:
lda_model.print_topics()

[(0,
  '0.009*"russian" + 0.009*"russia" + 0.009*"country" + 0.008*"government" + 0.007*"north" + 0.006*"state" + 0.006*"force" + 0.005*"minister" + 0.005*"attack" + 0.005*"security"'),
 (1,
  '0.010*"china" + 0.009*"percent" + 0.008*"european" + 0.007*"brexit" + 0.007*"trade" + 0.006*"country" + 0.006*"britain" + 0.006*"business" + 0.005*"market" + 0.005*"government"'),
 (2,
  '0.010*"player" + 0.008*"football" + 0.008*"sport" + 0.007*"berry" + 0.007*"kathrada" + 0.005*"ruffin" + 0.005*"world" + 0.005*"play" + 0.005*"award" + 0.004*"grenoble"'),
 (3,
  '0.013*"people" + 0.010*"woman" + 0.008*"police" + 0.008*"family" + 0.007*"child" + 0.006*"school" + 0.005*"find" + 0.005*"year" + 0.005*"young" + 0.004*"water"'),
 (4,
  '0.019*"trump" + 0.011*"house" + 0.010*"president" + 0.007*"committee" + 0.006*"court" + 0.006*"state" + 0.005*"white" + 0.005*"report" + 0.005*"people" + 0.005*"issue"')]

### How to interpret this?
The top 10 keywords that contribute to the topic are showcased with their respective weight.

Let's try to interpret the 5 topics:

- Topic 1: key words like "Russia", "Country", "Government", "Minister" suggest **Politics in Russia**
- Topic 2: key words like "China", "Brexit","Trade", "Business", "Market" suggest **Inter country trade news**
- Topic 3: key words like "Player","Sport","World" suggest **Sports news (football)**
- Topic 4: key words like "People","Woman","Police", "Family,"Child" suggest **Domestic news**
- Topic 5: key words like "Trump", "State", "White", "Committee" suggest **Polictics in USA**

### Compute Model Perplexity and Coherence Score

In [16]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = models.CoherenceModel(model=lda_model, texts=lemma_doc, dictionary=word2id, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.588727660028226

Coherence Score:  0.4507504883751433


Coherence measures the relative distance between words within a topic. There are two major types C_V typically 0 < x < 1 and uMass -14 < x < 14.
Coherence score of 0.4 is low. I want to explore what would have been the ideal number of topics. Will explore the elbow method below. 

In [17]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model=LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=42, update_every=1, chunksize=100, 
                     passes=10, alpha='auto', per_word_topics=True)
        model_list.append(model)
        coherencemodel = models.CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
#Use this to get the graph of optimal # of topics
model_list, coherence_values = compute_coherence_values(dictionary=word2id, corpus=corpus, texts=lemma_doc, start=2, limit=100, step=10)
# Show graph
import matplotlib.pyplot as plt
limit=100; start=2; step=10;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

You can decide on the number of topics based on this analysis. Note that the Customization used for 5 topic model (lda_model) and the optimization models is difference therefore the Coherence score for 5 topics LDA model differ.

### Vizualize the topics

In [18]:
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(lda_model,corpus,word2id)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4     -0.161832  0.022906       1        1  31.212732
0     -0.137963  0.110081       2        1  24.438366
1     -0.065682  0.052482       3        1  22.085140
3      0.020831 -0.266392       4        1  19.341375
2      0.344646  0.080924       5        1   2.922387, topic_info=      Category         Freq          Term        Total  loglift  logprob
term                                                                    
134    Default  5184.000000         trump  5184.000000  30.0000  30.0000
234    Default  2932.000000         house  2932.000000  29.0000  29.0000
2452   Default  1866.000000         china  1866.000000  28.0000  28.0000
841    Default  1988.000000       russian  1988.000000  27.0000  27.0000
840    Default  1943.000000        russia  1943.000000  26.0000  26.0000
352    Default  1986.000000         woman  1986.000000  25.0000  25.0000
2261   Default  1712.000000       percent  1712.000000  24.0000  24.0000
192    Default  1690.000000         court  1690.000000  23.0000  23.0000
863    Default  2027.000000      european  2027.000000  22.0000  22.0000
35     Default  1799.000000     committee  1799.000000  21.0000  21.0000
1989   Default  1252.000000         trade  1252.000000  20.0000  20.0000
214    Default  1733.000000        family  1733.000000  19.0000  19.0000
856    Default  1442.000000        brexit  1442.000000  18.0000  18.0000
100    Default  3630.000000     president  3630.000000  17.0000  17.0000
432    Default  4629.000000        people  4629.000000  16.0000  16.0000
29     Default  1411.000000         child  1411.000000  15.0000  15.0000
353    Default  1522.000000         world  1522.000000  14.0000  14.0000
378    Default  3632.000000       country  3632.000000  13.0000  13.0000
741    Default  2321.000000        police  2321.000000  12.0000  12.0000
1664   Default  1027.000000        market  1027.000000  11.0000  11.0000
857    Default  1332.000000       britain  1332.000000  10.0000  10.0000
116    Default  1019.000000        school  1019.000000   9.0000   9.0000
648    Default  1757.000000         north  1757.000000   8.0000   8.0000
172    Default  1263.000000      business  1263.000000   7.0000   7.0000
700    Default  1248.000000        attack  1248.000000   6.0000   6.0000
1782   Default  1021.000000  intelligence  1021.000000   5.0000   5.0000
1237   Default   897.000000      military   897.000000   4.0000   4.0000
351    Default  1566.000000         white  1566.000000   3.0000   3.0000
6862   Default   912.000000         korea   912.000000   2.0000   2.0000
798    Default   825.000000       company   825.000000   1.0000   1.0000
...        ...          ...           ...          ...      ...      ...
2658    Topic5    54.081562     instagram    55.019585   3.5156  -6.1431
34593   Topic5    51.653210     gymnastic    52.554897   3.5155  -6.1891
18178   Topic5    52.197594         queer    53.205891   3.5136  -6.1786
8785    Topic5    45.931786         album    46.821865   3.5136  -6.3065
4853    Topic5    44.440090       actress    45.331676   3.5129  -6.3395
12952   Topic5    44.076237     celebrity    44.969959   3.5127  -6.3477
25370   Topic5    44.443634     tottenham    45.359825   3.5124  -6.3394
11909   Topic5    45.868202     brazilian    46.824947   3.5121  -6.3078
1825    Topic5    41.705715       athlete    42.595432   3.5117  -6.4030
7064    Topic5    43.402985         kazan    44.339050   3.5114  -6.3631
44823   Topic5   107.425087       ronaldo   109.748528   3.5114  -5.4568
13766   Topic5    43.043354      kindness    43.986813   3.5111  -6.3714
44303   Topic5   133.617249        ruffin   137.194885   3.5063  -5.2386
37984   Topic5    68.035271        brooke    69.603226   3.5100  -5.9136
5421    Topic5    87.723808          film    92.415657   3.4807  -5.6594
2147    Topic5   196.995789         sport   216.997208   3.4361  -4.8504
44